In [1]:
import numpy as np
import pandas as pd
import scipy
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

# MLE and MAP

#### Load data

In [2]:
data_large = pd.read_csv("../data/poly_regression_large.csv")
X_large, Y_large = data_large["x"], data_large["y"]
X_train_large, X_test_large, Y_train_large, Y_test_large = train_test_split(X_large, Y_large, test_size=0.20, random_state=333, shuffle=True)
_, _, Y_train_large, Y_test_large = np.array(X_train_large), np.array(X_test_large), np.array(Y_train_large), np.array(Y_test_large)

data_small = pd.read_csv("../data/poly_regression_small.csv")
X_small, Y_small = data_small["x"], data_small["y"]

#### Optimization functions

In [7]:
def SGD(loss, grad_loss, D, theta0, alpha, batch_size, n_epochs):
    X, y = D  # Unpack the data
    d = theta0.shape[0] # While theta0 has shape (d, )
    idx = np.arange(0, N) # This is required for the shuffling

    # Initialization of history vectors
    theta_history = np.zeros((n_epochs, d))  # Save parameters at each epoch
    loss_history = np.zeros((n_epochs, ))  # Save loss values at each epoch
    grad_norm_history = np.zeros((n_epochs, ))  # Save gradient norms at each epoch
    
    # Initialize weights
    theta = theta0
    for epoch in range(n_epochs):
        # Shuffle the data at the beginning of each epoch
        np.random.shuffle(idx)
        X = X[idx]
        y = y[idx]

        # Initialize a vector that saves the gradient of the loss at each iteration
        grad_loss_vec = []

        for batch_start in range(0, N, batch_size):
            batch_end = min(batch_start + batch_size, N)
            X_batch = X[batch_start:batch_end]
            y_batch = y[batch_start:batch_end]
            
            # Compute the gradient of the loss
            gradient = grad_loss(theta, X_batch, y_batch)
            grad_loss_vec.append(np.linalg.norm(gradient, 2))

            # Update weights
            theta = theta - alpha * gradient

        # Save the updated values
        theta_history[epoch] = theta
        loss_history[epoch] = loss(theta, X, y)
        grad_norm_history[epoch] = np.mean(grad_loss_vec)
    
    return theta_history, loss_history, grad_norm_history

In [8]:
def GD(loss, grad_loss, D, theta0, alpha, n_epochs):
    X, y = D  # Unpack the data
    N = X.shape[0] # We assume both X and Y has shape (N, )
    d = theta0.shape[0] # While theta0 has shape (d, )
    idx = np.arange(0, N) # This is required for the shuffling

    # Initialization of history vectors
    theta_history = np.zeros((n_epochs, d))  # Save parameters at each epoch
    loss_history = np.zeros((n_epochs, ))  # Save loss values at each epoch
    grad_norm_history = np.zeros((n_epochs, ))  # Save gradient norms at each epoch
    
    # Initialize weights
    theta = theta0
    for epoch in range(n_epochs):
        # Compute the gradient of the loss
        gradient = grad_loss(theta, X, y)
        grad_norm_history[epoch] = np.linalg.norm(gradient, 2)

        # Update weights
        theta = theta - alpha * gradient

        # Save the updated values
        theta_history[epoch] = theta
        loss_history[epoch] = loss(theta, X, y)
    
    return theta_history, loss_history, grad_norm_history

In [9]:

def normalEquations(K, D):
    x, y = D
    N = x.shape[0]
    PhiX = vandermonde(x, K)
    # solve X XT theta = X Y
    L = np.linalg.cholesky(PhiX @ PhiX.T)
    # solve L z = X Y
    z = np.linalg.solve(L, PhiX @ y)
    # solve LT theta = z
    theta = np.linalg.solve(L.T, z)
    return theta

def vandermonde(x, K):
    v = np.ones((K, N))
    print(N)
    print(x.shape)
    print(v.shape)
    for i in range(1, K):
        v[i] = x ** i
    return v


### Compute the MLE

In [11]:
K = 5
N = X_small.shape[0]
batch_size = 5
n_epochs = 100
theta0 = np.zeros((K,1))
alpha = 0.1

# MLE
def loss(theta, x, y):
    # return (1/2) * np.linalg.norm((vandermonde(x, K) @ theta - y)**2, 2)
    return np.square(np.linalg.norm(x.T @ ((x @ theta) - y), 2)) / N

def grad_loss(theta, x, y):
    # N = len(y)
    # y_pred = f(x, y, theta)
    # error = y_pred - y
    
    # gradient = np.zeros_like(theta)
    # for i in range(len(theta)):
    #     gradient[i] = 2 * np.sum(error * (x ** i)) / N
    # return gradient
    
    gradient = np.zeros_like(theta)
    for i in range(len(theta)):
        gradient[i] = (2/N) * np.linalg.norm((x.T @ ((x @ theta[i]) - y)), 2) / N
    
    return gradient

theta_SGD = SGD(loss, grad_loss, (X_small, Y_large), theta0, alpha, batch_size, n_epochs)
theta_GD = ...
theta_NE = ...

Exception: Dot product shape mismatch, (5,) vs (1,)

### Compute the relative error

In [20]:
err_SGD = (1/ len(Y_test_large)) * sum([loss(theta_SGD, x, y) for x, y in list(zip(X_test_large, Y_test_large))])
err_GD = ...
err_NE = ...

# err_M_s = (1/ len(Y_test_large)) * sum([loss(theta, x, y) for x, y in list(zip(X_test_large, Y_test_large))])
print(err_SGD)

IndexError: too many indices for array: array is 1-dimensional, but 2 were indexed